In [17]:
# pip install playwright

In [18]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

In [25]:
async def scrape_jobstreet():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # Visible for CAPTCHA
        page = await browser.new_page()
        await page.goto("https://ph.jobstreet.com/Data-Science-jobs?jobId=85753657&type=standard")

        print("🛑 Manually solve CAPTCHA in browser window if needed...")
        await asyncio.sleep(30)

        jobs = await page.query_selector_all('article[data-testid="job-card"]')
        print(f"🔍 Found {len(jobs)} job listings")

        data = []

        for job in jobs:
            try:
                anchor = await job.query_selector("a[data-automation='job-list-item-link-overlay']")
                url_suffix = await anchor.get_attribute("href") if anchor else "N/A"
                job_url = "https://ph.jobstreet.com" + url_suffix if url_suffix and url_suffix.startswith("/job") else url_suffix

                # Fallback: extract all span texts
                all_spans = await job.query_selector_all("span")
                text_items = []
                for span in all_spans:
                    try:
                        txt = await span.inner_text()
                        if txt.strip():
                            text_items.append(txt.strip())
                    except:
                        continue

                title = text_items[0] if len(text_items) > 0 else "N/A"
                company = text_items[1] if len(text_items) > 1 else "N/A"
                location = text_items[2] if len(text_items) > 2 else "N/A"
                salary = next((t for t in text_items if "₱" in t or "PHP" in t), "N/A")
                posted = next((t for t in text_items if "ago" in t), "N/A")

                data.append({
                    "Title": title,
                    "Company": company,
                    "Location": location,
                    "Salary": salary,
                    "Posted": posted,
                    "URL": job_url
                })

            except Exception as e:
                print("⚠️ Error parsing a job:", e)

        await browser.close()
        return pd.DataFrame(data)

In [26]:
# Run the async function inside Jupyter
df = await scrape_jobstreet()
df.head()

🛑 Manually solve CAPTCHA in browser window if needed...
🔍 Found 32 job listings


,Title,Company,Location,Salary,Posted,URL
0,at\nNational Data and Surveying Services Inc,at,"Pasig City, Metro Manila","₱25,000 – ₱35,000 per month",1d ago,https://ph.jobstreet.com/job/85753657?type=sta...
1,"at\nHelloConnect, Inc.",at,"Quezon City, Metro Manila(Remote)","₱75,000 – ₱100,000 per month",10h ago,https://ph.jobstreet.com/job/85795350?type=sta...
2,"at\nRiteMed Phils., Inc.",at,"Mandaluyong City, Metro Manila(Hybrid)",N/A,1d ago,https://ph.jobstreet.com/job/85756366?type=sta...
3,at\nGenpact Services LLC,at,"Muntinlupa City, Metro Manila",N/A,23h ago,https://ph.jobstreet.com/job/85782442?type=sta...
4,at\nTHOMSON REUTERS CORP PTE LTD - PHILIPPINE ...,at,"Taguig City, Metro Manila",N/A,8d ago,https://ph.jobstreet.com/job/85550130?type=sta...


In [27]:
display(df)

,Title,Company,Location,Salary,Posted,URL
0,at\nNational Data and Surveying Services Inc,at,"Pasig City, Metro Manila","₱25,000 – ₱35,000 per month",1d ago,https://ph.jobstreet.com/job/85753657?type=sta...
1,"at\nHelloConnect, Inc.",at,"Quezon City, Metro Manila(Remote)","₱75,000 – ₱100,000 per month",10h ago,https://ph.jobstreet.com/job/85795350?type=sta...
2,"at\nRiteMed Phils., Inc.",at,"Mandaluyong City, Metro Manila(Hybrid)",N/A,1d ago,https://ph.jobstreet.com/job/85756366?type=sta...
3,at\nGenpact Services LLC,at,"Muntinlupa City, Metro Manila",N/A,23h ago,https://ph.jobstreet.com/job/85782442?type=sta...
4,at\nTHOMSON REUTERS CORP PTE LTD - PHILIPPINE ...,at,"Taguig City, Metro Manila",N/A,8d ago,https://ph.jobstreet.com/job/85550130?type=sta...
5,at\nMicroSourcing,at,Metro Manila(Remote),"₱120,000 per month",2d ago,https://ph.jobstreet.com/job/85735664?type=sta...
6,at\nTransUnion Information Solutions Inc.,at,"Makati City, Metro Manila(Hybrid)",N/A,7d ago,https://ph.jobstreet.com/job/85603671?type=sta...
7,"at\nCover Desk Philippines, Inc.",at,"Quezon City, Metro Manila","₱30,000 – ₱35,000 per month",7d ago,https://ph.jobstreet.com/job/85593540?type=sta...
8,at\nEY GLOBAL DELIVERY SERVICES (GDS) PHILIPPINES,at,Metro Manila(Hybrid),N/A,3d ago,https://ph.jobstreet.com/job/85700899?type=sta...
9,at\nSecurity Bank Corporation,at,"Makati City, Metro Manila(Hybrid)",N/A,10h ago,https://ph.jobstreet.com/job/85795523?type=sta...
